In [30]:
import os
import pandas as pd

In [31]:
from toposort import toposort, toposort_flatten

In [32]:
from collections import defaultdict 

In [33]:
path = "./"

In [34]:
folder_list = [each for each in list(os.walk(path))[0][1] if "python" in each]

In [35]:
variables_to_look = ['depends','makedepends']

In [36]:
def variable_or_not(line):
    
    if "=" in line and len(line.strip().split("=")[0].split(" "))==1:
        return each_line.strip().split("=")[0]
    else:
        return False
    

In [37]:
#Class to represent a graph 
class Graph: 
    def __init__(self,vertices): 
        self.graph = defaultdict(list) #dictionary containing adjacency List 
        self.V = vertices #No. of vertices 
  
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
  
    # A recursive function used by topologicalSort 
    def topologicalSortUtil(self,v,visited,stack): 
  
        # Mark the current node as visited. 
        visited[v] = True
  
        # Recur for all the vertices adjacent to this vertex 
        for i in self.graph[v]: 
            if visited[i] == False: 
                self.topologicalSortUtil(i,visited,stack) 
  
        # Push current vertex to stack which stores result 
        stack.insert(0,v) 
  
    # The function to do Topological Sort. It uses recursive  
    # topologicalSortUtil() 
    def topologicalSort(self): 
        # Mark all the vertices as not visited 
        visited = [False]*self.V 
        stack =[] 
  
        # Call the recursive helper function to store Topological 
        # Sort starting from all vertices one by one 
        for i in range(self.V): 
            if visited[i] == False: 
                self.topologicalSortUtil(i,visited,stack) 
  
        # Print contents of the stack 
        print(stack)


In [38]:
# start reading the dependencies

flag = False
main_depends = {}
for each_folder in folder_list:
    with open(path+each_folder+"/PKGBUILD",'r') as file:
        temp = file.readlines()
        
        
        stack = []
        for each in temp:

            
            if each.split("=")[0] in variables_to_look:
                flag = True
            
            if flag == True:
                
                stack.append(each.strip().replace("'",""))
                #print(each.strip().replace("'",""))
                
            if ')' in each.strip():
                flag = False
                if len(stack) > 0:
                    main_depends[each_folder] = stack
                
            
        
for each in list(main_depends.keys()):
    
    temp = []
    for x in main_depends[each]:
        temp.append(x.split("=")[-1])
        
    main_depends[each] = set([each for each in " ".join(temp).replace("(","").replace(")","").split(" ") if len(each)>0])


In [39]:
main_modules = list(main_depends.keys())
all_modules = list(set([j for i in list(main_depends.values()) for j in i]+main_modules))

In [40]:
main_frame = pd.DataFrame(columns=all_modules,index=all_modules).fillna(0)

In [41]:
for ind in list(main_frame.index):
    for col in list(main_frame.columns):
        if ind in main_modules and col!=ind:
            
            for each in main_depends[ind]:
                main_frame.loc[ind,each] = 1
        
main_frame

,python-setuptools,python-dog,python-banana,python,python-cactus,python-apple,python-elephant
python-setuptools,0,0,0,0,0,0,0
python-dog,0,0,0,1,0,0,1
python-banana,0,0,0,1,0,1,0
python,0,0,0,0,0,0,0
python-cactus,0,0,1,1,0,0,0
python-apple,1,0,0,1,0,0,0
python-elephant,0,0,0,1,0,1,0


In [43]:
toposort_flatten(main_depends)

['python',
 'python-setuptools',
 'python-apple',
 'python-banana',
 'python-elephant',
 'python-cactus',
 'python-dog']

In [52]:
# circular dependecies
data = {
    
1:{2},
2:{1},
3:{4}
}

toposort_flatten(data)

CircularDependencyError: Circular dependencies exist among these items: {1:{2}, 2:{1}}